Problem Statment: We need to read the textual story and produce background music at appropriate time related to story's context and with modulation\


In this Superimposition model we will build like 



then this particular sound is played until it is superimposed with other sounds..here is a catch that maybe we want to play the current audio as well as the new audio 



so what exactly i have planned is 



i have a superimposition model which can generate a specific type of sound say enviroment, emotional bgms etc... we will superimpose sounds from this specialist model with proper smoothening and modulation and play each type of audio at correct time according to story's context.....

So like say i define a reading speed to be 1 word /sec ..meaning i read one word per second...and i define a minimum duration say dt




so we may define a struct like 

audio_to_play , start_time, weigth, duration dt

and say audio_class_to_play = array of above struct 

so lets define our tools : 



Tool 1:

play_audio(  audio_classes_to_play )

It smoothens and plays the audio 

like it synthesis audio like 

iterate in array for each audio ...{ 

till start do play no audio ..then after start time...slowly start the audio (ie first amplitude is less then graudally increases) .. and continues till dt and the faints away by decrasing amplitude slowly ...also play audio should be according to wegiht of audio

}

and we combine all the audio clips and play a superimposed sound



Tool2:

decice audio

it gets what audio it needs to play ..like Enviormental Sounds, emotional background music, Movie Bgms etc and tell speicalist model to generate that type of sounds 

say we define models:
emotional_musicGenerator : emotional music specialist (assume you have this specialist)
enviroment_musicGenerator : specialist model (assume you have this)

and it gets the story context ..and according to position of word of sound class it generate audio_class_to_play array by passing start time of audio, audio from the specialist model ,..a default duration of 2 sec, and weight based on context (from words like loud, faint etc)

you may use some function like
DEFAULT_WEIGHTS = 0.5
LOUD_WEIGHT = 0.8
LITTLE_WEIGHT = 0.2

# words
LOUD = "loud"
LITTLE="little"

def get_audio_classes_with_weights(prompt):
    """Parses the prompt to extract class names and their associated weights."""
    words = prompt.lower().split()
    sound_of_class = []
    prev_word_weight = DEFAULT_WEIGHTS
    sum_of_weights = 0    
    i = 0

    while i < len(words):
        word_matched = False
        
        # First, try to match 2-word combinations (for classes like "dog_bark" -> "dog bark")
        if i < len(words) - 1:
            two_word_phrase = f"{words[i]} {words[i+1]}"
            for class_name in classes:
                if two_word_phrase == class_name.lower().replace("_", " "):
                    sound_of_class.append((class_name, prev_word_weight))
                    sum_of_weights += prev_word_weight
                    word_matched = True
                    i += 2  # Skip both words since we matched them as a pair
                    break
        
        # If no 2-word match found, try single word
        if not word_matched:
            current_word = words[i]
            
            # Check if it's a single-word class
            for class_name in classes:
                if current_word == class_name.replace("_", " "):
                    sound_of_class.append((class_name, prev_word_weight))
                    sum_of_weights += prev_word_weight
                    word_matched = True
                    break
            
            # Check if it's a weight modifier word
            if current_word == LOUD:
                prev_word_weight = LOUD_WEIGHT
            elif current_word == LITTLE:
                prev_word_weight = LITTLE_WEIGHT
            else:
                prev_word_weight = DEFAULT_WEIGHTS
                
            i += 1  # Move to next word

    print(f"[DEBUG] Sound of classes and their weights: {sound_of_class}")
    return sound_of_class, sum_of_weights    
    
# test
prompt = " dog barking and little bird chirping loud cat meowing "
classes_with_weights, total_weight = get_audio_classes_with_weights(prompt)
print(f"Classes with weights: {classes_with_weights}")
print(f"Total weight: {total_weight}")

output:
[DEBUG] Sound of classes and their weights: [('Dog_Barking', 0.5), ('bird_chirping', 0.2), ('Cat_Meowing', 0.8)]
Classes with weights: [('Dog_Barking', 0.5), ('bird_chirping', 0.2), ('Cat_Meowing', 0.8)]
Total weight: 1.5




give me python functions and code
also duration is something we need to look on since emotional music should last longer, enviroment music should last smaller (say when some other audio take over the current enviromental sound)

so for example for a textual story input : Suddenly rain started so i ran to shelter where i heard loud dog barking
and suppose reading_speed = 2 words per second
default_duration = 2 sec
so our model should generate audio like  [<rain>, starttime=0.5 second, duration : default(2 sec) , weight:default=0.5 ],
[<running>, starttime=2.5, second, duration : default(2 sec) , weight:default=0.5 ],
[<dog bark>, starttime=6, second, duration : default(2 sec) , weight:loud=0.8 ]

you may make maths tools to calculate starttime ...

suggest me is this framework good or you have any better framework? if yes then give me functions based python code to implement above with debug statments



In [5]:
import math
from dataclasses import dataclass, field
from typing import List, Dict, Tuple

# --- (For a real app, you would: import pydub) ---
# from pydub import AudioSegment
# We will "mock" pydub by just printing what we would do,
# since we can't generate real audio here.

# ==============================================================================
# 1. DATA STRUCTURES (Your "Struct")
# ==============================================================================

# Using a dataclass, which is the modern Python equivalent of a struct
@dataclass
class AudioCue:
    """Stores all information needed for a single sound event."""
    audio_prompt: str      # Prompt to send to the specialist (e.g., "rain", "dog bark")
    start_time_ms: int     # When the sound should start
    duration_ms: int       # How long the sound should play
    weight_db: float       # Volume adjustment in decibels (dB)
    audio_type: str        # 'SFX', 'AMBIENCE', or 'MUSIC'
    fade_ms: int = 500     # Default fade in/out time

# ==============================================================================
# 2. CONFIGURATION & KEYWORDS (Your "Tools")
# ==============================================================================

# --- Reading Speed Configuration ---
READING_SPEED_WPS = 2.0  # Words Per Second

# --- Duration Configuration ---
DEFAULT_SFX_DURATION_MS = 2000  # 2 seconds (for short SFX)
DEFAULT_FADE_MS = 500           # 500ms fade in/out for most sounds

# --- Weight/Volume Configuration (Your LOUD/LITTLE logic) ---
# We use decibels (dB) for volume. 0 is normal.
# +6dB is roughly 2x louder, -6dB is roughly 2x quieter.
DEFAULT_WEIGHT_DB = 0.0
LOUD_WEIGHT_DB = 6.0
FAINT_WEIGHT_DB = -6.0

MODIFIER_WORDS = {
    "loud": LOUD_WEIGHT_DB,
    "faint": FAINT_WEIGHT_DB,
    "little": FAINT_WEIGHT_DB,
    "soft": FAINT_WEIGHT_DB,
    "quiet": FAINT_WEIGHT_DB,
    "roaring": LOUD_WEIGHT_DB,
    "blaring": LOUD_WEIGHT_DB,
}

# --- Keyword to Specialist Mapping ---
# This dictionary maps keywords to (audio_prompt, specialist_type)
# We handle multi-word keys (like "dog barking") by checking them first.
SOUND_KEYWORDS: Dict[str, Tuple[str, str]] = {
    # SFX (Short)
    "dog barking": ("dog bark", "SFX"),
    "barking": ("dog bark", "SFX"),
    "ran": ("running footsteps", "SFX"),
    "running": ("running footsteps", "SFX"),
    "footsteps": ("footsteps", "SFX"),
    "door creaked": ("door creak", "SFX"),
    "creaked": ("door creak", "SFX"),
    "shout": ("shout", "SFX"),
    "screamed": ("scream", "SFX"),
    "laughed": ("laughing", "SFX"),

    # AMBIENCE (Longer, environmental)
    "rain": ("heavy rain", "AMBIENCE"),
    "storm": ("thunder storm", "AMBIENCE"),
    "wind": ("howling wind", "AMBIENCE"),
    "forest": ("forest sounds", "AMBIENCE"),
    "city": ("city traffic", "AMBIENCE"),
    "shelter": ("rain on a roof", "AMBIENCE"), # Contextual!
    
    # MUSIC (Longer, emotional)
    "suddenly": ("suspenseful stinger", "MUSIC"), # "Stinger" is a short music cue
    "sad": ("sad violin", "MUSIC"),
    "happy": ("upbeat happy music", "MUSIC"),
    "felt a chill": ("eerie suspense music", "MUSIC"),
    "scared": ("scary horror music", "MUSIC"),
    "emotional": ("emotional piano", "MUSIC"),
}

# ==============================================================================
# 3. MOCKED SPECIALIST MODELS (Your "Generators")
# ==============================================================================
# In a real app, these would call AI models (like AudioGen, MusicGen)
# and return a pydub.AudioSegment object.

def sfx_generator(prompt: str, duration_ms: int):
    """MOCK: Generates a short sound effect."""
    print(f"    [MOCK SFX_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    # --- Real code would be: ---
    # audio_data = call_audiogen_api(prompt, duration_ms)
    # return AudioSegment(data=audio_data, ...)
    return f"[MOCK_AUDIO: {prompt}]"

def environment_generator(prompt: str, duration_ms: int):
    """MOCK: Generates an ambient environmental sound."""
    print(f"    [MOCK ENV_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    return f"[MOCK_AUDIO: {prompt}]"

def emotional_music_generator(prompt: str, duration_ms: int):
    """MOCK: Generates a background music track."""
    print(f"    [MOCK MUSIC_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    return f"[MOCK_AUDIO: {prompt}]"

SPECIALIST_MAP = {
    "SFX": sfx_generator,
    "AMBIENCE": environment_generator,
    "MUSIC": emotional_music_generator,
}

# ==============================================================================
# 4. TOOL 2: decide_audio (The Scheduler / NLP Parser)
# ==============================================================================
# This is the "brain" of the operation. It reads the story and generates
# the list of AudioCues.

def decide_audio(story_text: str, speed_wps: float) -> List[AudioCue]:
    """
    Parses the story text and creates a timed list of AudioCues.
    """
    print(f"[DECIDER] Starting audio decision process...")
    print(f"[DECIDER] Reading Speed: {speed_wps} words/sec")

    words = story_text.lower().split()
    total_words = len(words)
    total_duration_ms = math.ceil((total_words / speed_wps) * 1000)
    
    cues_to_play: List[AudioCue] = []
    
    # --- Sort keywords: multi-word first ---
    # This is crucial so we match "dog barking" before "barking"
    sorted_keywords = sorted(SOUND_KEYWORDS.keys(), key=len, reverse=True)
    
    current_weight_db = DEFAULT_WEIGHT_DB
    last_cue_time: Dict[str, int] = {"SFX": -1, "AMBIENCE": -1, "MUSIC": -1}
    last_cue_index: Dict[str, int] = {"SFX": -1, "AMBIENCE": -1, "MUSIC": -1}

    i = 0
    while i < total_words:
        current_time_ms = math.ceil((i / speed_wps) * 1000)
        word = words[i]
        
        # --- 1. Check for weight modifiers ---
        if word in MODIFIER_WORDS:
            current_weight_db = MODIFIER_WORDS[word]
            print(f"[DECIDER] Word '{word}' at {current_time_ms}ms. Setting weight to: {current_weight_db}dB")
            i += 1
            continue # Go to next word

        # --- 2. Check for sound keywords (multi-word first) ---
        matched_keyword = None
        for keyword in sorted_keywords:
            kw_len = len(keyword.split())
            if i + kw_len <= total_words:
                phrase = " ".join(words[i : i + kw_len])
                if phrase == keyword:
                    matched_keyword = keyword
                    break
        
        if matched_keyword:
            audio_prompt, audio_type = SOUND_KEYWORDS[matched_keyword]
            
            print(f"[DECIDER] Keyword '{matched_keyword}' ({audio_type}) at {current_time_ms}ms.")

            # --- 3. Create the AudioCue ---
            cue = AudioCue(
                audio_prompt=audio_prompt,
                start_time_ms=current_time_ms,
                duration_ms=DEFAULT_SFX_DURATION_MS, # Default, will be changed for AMBIENCE/MUSIC
                weight_db=current_weight_db,
                audio_type=audio_type
            )
            
            # --- 4. Handle "Smart Duration" for AMBIENCE/MUSIC ---
            # We need to update the *previous* cue of the same type to stop it.
            if audio_type == "AMBIENCE" or audio_type == "MUSIC":
                last_idx = last_cue_index[audio_type]
                if last_idx != -1:
                    # Found a previous cue. Let's update its duration.
                    prev_cue = cues_to_play[last_idx]
                    prev_cue.duration_ms = current_time_ms - prev_cue.start_time_ms
                    print(f"    [DECIDER] Updating previous '{prev_cue.audio_type}' cue to end at {current_time_ms}ms.")

                # For this new cue, set its duration to play until the end (for now)
                
                cue.duration_ms = total_duration_ms - current_time_ms
                last_cue_index[audio_type] = len(cues_to_play) # Store index of this cue
            
            cues_to_play.append(cue)
            
            # Reset weight after using it
            current_weight_db = DEFAULT_WEIGHT_DB
            i += len(matched_keyword.split()) # Skip the words we just matched
        
        else:
            i += 1 # No match, move to the next word

    print(f"[DECIDER] Finished parsing. Found {len(cues_to_play)} audio cues.")
    return cues_to_play, total_duration_ms

# ==============================================================================
# 5. TOOL 1: play_audio (The Mixer / Superimposition)
# ==============================================================================
# This is the "muscle" of the operation. It takes the AudioCue list
# and uses pydub to mix them all together.

def play_audio(audio_cues: List[AudioCue], total_duration_ms: int):
    """
    Generates and mixes all audio cues into a single track (MOCKED).
    """
    print("\n[MIXER] Starting audio mixing process...")
    print(f"[MIXER] Creating silent audio canvas of {total_duration_ms}ms.")

    # --- Real Pydub Code ---
    # final_score = AudioSegment.silent(duration=total_duration_ms)
    # --- Mock Code ---
    final_score_log = [f"SILENT_CANVAS ({total_duration_ms}ms)"]


    for cue in audio_cues:
        print(f"[MIXER] Processing cue: '{cue.audio_prompt}' ({cue.audio_type})")

        # --- 1. Call Specialist Model (Mocked) ---
        specialist_func = SPECIALIST_MAP[cue.audio_type]
        mock_audio_data = specialist_func(cue.audio_prompt, cue.duration_ms)

        # --- 2. Process with Pydub (Mocked) ---
        # In real code, mock_audio_data would be an AudioSegment object.
        print(f"    [MIXER] Applying fades and volume:")
        print(f"    ... Fading In: {cue.fade_ms}ms, Fading Out: {cue.fade_ms}ms")
        print(f"    ... Adjusting Volume: {cue.weight_db}dB")

        # --- Real Pydub Code ---
        # fade_time = min(cue.fade_ms, cue.duration_ms // 2) # Don't fade longer than half the clip
        # audio_clip = mock_audio_data
        # processed_clip = audio_clip.fade_in(fade_time).fade_out(fade_time)
        # processed_clip = processed_clip + cue.weight_db # Apply volume
        
        print(f"    [MIXER] Overlaying '{cue.audio_prompt}' at {cue.start_time_ms}ms.")
        
        # --- Real Pydub Code ---
        # final_score = final_score.overlay(processed_clip, position=cue.start_time_ms)
        
        # --- Mock Code ---
        final_score_log.append(
            f"  + OVERLAY: '{cue.audio_prompt}' at {cue.start_time_ms}ms "
            f"(Duration: {cue.duration_ms}ms, Volume: {cue.weight_db}dB)"
        )

    print("\n[MIXER] === FINAL AUDIO SCORE ASSEMBLED (MOCK) ===")
    for line in final_score_log:
        print(line)
    
    # --- Real Pydub Code ---
    # print("[MIXER] Exporting to final_score.mp3...")
    # final_score.export("final_score.mp3", format="mp3")
    # print("[MIXER] Done.")
    # return final_score
    
    return final_score_log

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================

if __name__ == "__main__":
    
    # Your example story
    story = "Suddenly rain started so i ran to shelter where i heard loud dog barking"
    
    print("="*40)
    print(f"Processing Story: \"{story}\"")
    print("="*40 + "\n")

    # --- Run Tool 2: Decide Audio ---
    cues, total_duration = decide_audio(story, READING_SPEED_WPS)
    
    print("\n--- Generated Audio Cues ---")
    if not cues:
        print("No audio cues found.")
    for c in cues:
        print(f"  - {c}")
    print(f"Total Story Duration: {total_duration}ms")
    print("-" * 30 + "\n")
    
    # --- Run Tool 1: Play Audio ---
    play_audio(cues, total_duration)

    print("\n" + "="*40)
    print("Example 2: A more complex story")
    print("="*40 + "\n")

    story_2 = ("It was a happy day. I was walking in the forest when suddenly "
               "i felt a chill. I heard a faint dog barking. Then a loud shout!")

    cues_2, total_duration_2 = decide_audio(story_2, READING_SPEED_WPS)
    
    print("\n--- Generated Audio Cues ---")
    for c in cues_2:
        print(f"  - {c}")
    print(f"Total Story Duration: {total_duration_2}ms")
    print("-" * 30 + "\n")
    
    play_audio(cues_2, total_duration_2)

Processing Story: "Suddenly rain started so i ran to shelter where i heard loud dog barking"

[DECIDER] Starting audio decision process...
[DECIDER] Reading Speed: 2.0 words/sec
[DECIDER] Keyword 'suddenly' (MUSIC) at 0ms.
[DECIDER] Keyword 'rain' (AMBIENCE) at 500ms.
[DECIDER] Keyword 'ran' (SFX) at 2500ms.
[DECIDER] Keyword 'shelter' (AMBIENCE) at 3500ms.
    [DECIDER] Updating previous 'AMBIENCE' cue to end at 3500ms.
[DECIDER] Word 'loud' at 5500ms. Setting weight to: 6.0dB
[DECIDER] Keyword 'dog barking' (SFX) at 6000ms.
[DECIDER] Finished parsing. Found 5 audio cues.

--- Generated Audio Cues ---
  - AudioCue(audio_prompt='suspenseful stinger', start_time_ms=0, duration_ms=7000, weight_db=0.0, audio_type='MUSIC', fade_ms=500)
  - AudioCue(audio_prompt='heavy rain', start_time_ms=500, duration_ms=3000, weight_db=0.0, audio_type='AMBIENCE', fade_ms=500)
  - AudioCue(audio_prompt='running footsteps', start_time_ms=2500, duration_ms=2000, weight_db=0.0, audio_type='SFX', fade_ms=500)

In [6]:
import math
from dataclasses import dataclass, field
from typing import List, Dict, Tuple

# --- (For a real app, you would: import pydub) ---
# from pydub import AudioSegment
# We will "mock" pydub by just printing what we would do,
# since we can't generate real audio here.

# ==============================================================================
# 1. DATA STRUCTURES (Your "Struct")
# ==============================================================================

# Using a dataclass, which is the modern Python equivalent of a struct
@dataclass
class AudioCue:
    """Stores all information needed for a single sound event."""
    audio_prompt: str      # Prompt to send to the specialist (e.g., "rain", "dog bark")
    start_time_ms: int     # When the sound should start
    duration_ms: int       # How long the sound should play
    weight_db: float       # Volume adjustment in decibels (dB)
    audio_type: str        # 'SFX', 'AMBIENCE', or 'MUSIC'
    fade_ms: int = 500     # Default fade in/out time

# ==============================================================================
# 2. CONFIGURATION & KEYWORDS (Your "Tools")
# ==============================================================================

# --- Reading Speed Configuration ---
READING_SPEED_WPS = 2.0  # Words Per Second

# --- Duration Configuration ---
DEFAULT_SFX_DURATION_MS = 2000  # 2 seconds (for short SFX)
DEFAULT_FADE_MS = 500           # 500ms fade in/out for most sounds

# --- Weight/Volume Configuration (Your LOUD/LITTLE logic) ---
# We use decibels (dB) for volume. 0 is normal.
# +6dB is roughly 2x louder, -6dB is roughly 2x quieter.
DEFAULT_WEIGHT_DB = 0.0
LOUD_WEIGHT_DB = 6.0
FAINT_WEIGHT_DB = -6.0

MODIFIER_WORDS = {
    "loud": LOUD_WEIGHT_DB,
    "faint": FAINT_WEIGHT_DB,
    "little": FAINT_WEIGHT_DB,
    "soft": FAINT_WEIGHT_DB,
    "quiet": FAINT_WEIGHT_DB,
    "roaring": LOUD_WEIGHT_DB,
    "blaring": LOUD_WEIGHT_DB,
}

# --- Keyword to Specialist Mapping ---
# This dictionary maps keywords to (audio_prompt, specialist_type)
# We handle multi-word keys (like "dog barking") by checking them first.
SOUND_KEYWORDS: Dict[str, Tuple[str, str]] = {
    # SFX (Short)
    "dog barking": ("dog bark", "SFX"),
    "barking": ("dog bark", "SFX"),
    "ran": ("running footsteps", "SFX"),
    "running": ("running footsteps", "SFX"),
    "footsteps": ("footsteps", "SFX"),
    "door creaked": ("door creak", "SFX"),
    "creaked": ("door creak", "SFX"),
    "shout": ("shout", "SFX"),
    "screamed": ("scream", "SFX"),
    "laughed": ("laughing", "SFX"),

    # AMBIENCE (Longer, environmental)
    "rain": ("heavy rain", "AMBIENCE"),
    "storm": ("thunder storm", "AMBIENCE"),
    "wind": ("howling wind", "AMBIENCE"),
    "forest": ("forest sounds", "AMBIENCE"),
    "city": ("city traffic", "AMBIENCE"),
    "shelter": ("rain on a roof", "AMBIENCE"), # Contextual!
    
    # MUSIC (Longer, emotional)
    "suddenly": ("suspenseful stinger", "MUSIC"), # "Stinger" is a short music cue
    "sad": ("sad violin", "MUSIC"),
    "happy": ("upbeat happy music", "MUSIC"),
    "felt a chill": ("eerie suspense music", "MUSIC"),
    "scared": ("scary horror music", "MUSIC"),
    "emotional": ("emotional piano", "MUSIC"),
}

# ==============================================================================
# 3. MOCKED SPECIALIST MODELS (Your "Generators")
# ==============================================================================
# In a real app, these would call AI models (like AudioGen, MusicGen)
# and return a pydub.AudioSegment object.

def sfx_generator(prompt: str, duration_ms: int):
    """MOCK: Generates a short sound effect."""
    print(f"    [MOCK SFX_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    # --- Real code would be: ---
    # audio_data = call_audiogen_api(prompt, duration_ms)
    # return AudioSegment(data=audio_data, ...)
    return f"[MOCK_AUDIO: {prompt}]"

def environment_generator(prompt: str, duration_ms: int):
    """MOCK: Generates an ambient environmental sound."""
    print(f"    [MOCK ENV_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    return f"[MOCK_AUDIO: {prompt}]"

def emotional_music_generator(prompt: str, duration_ms: int):
    """MOCK: Generates a background music track."""
    print(f"    [MOCK MUSIC_GENERATOR] Generating: '{prompt}' ({duration_ms}ms)")
    return f"[MOCK_AUDIO: {prompt}]"

SPECIALIST_MAP = {
    "SFX": sfx_generator,
    "AMBIENCE": environment_generator,
    "MUSIC": emotional_music_generator,
}

# ==============================================================================
# 4. TOOL 2: decide_audio (The Scheduler / NLP Parser)
# ==============================================================================
# This is the "brain" of the operation. It reads the story and generates
# the list of AudioCues.

def decide_audio(story_text: str, speed_wps: float) -> List[AudioCue]:
    """
    Parses the story text and creates a timed list of AudioCues.
    """
    print(f"[DECIDER] Starting audio decision process...")
    print(f"[DECIDER] Reading Speed: {speed_wps} words/sec")

    words = story_text.lower().split()
    total_words = len(words)
    total_duration_ms = math.ceil((total_words / speed_wps) * 1000)
    
    cues_to_play: List[AudioCue] = []
    
    # --- Sort keywords: multi-word first ---
    # This is crucial so we match "dog barking" before "barking"
    sorted_keywords = sorted(SOUND_KEYWORDS.keys(), key=len, reverse=True)
    
    current_weight_db = DEFAULT_WEIGHT_DB
    last_cue_time: Dict[str, int] = {"SFX": -1, "AMBIENCE": -1, "MUSIC": -1}
    last_cue_index: Dict[str, int] = {"SFX": -1, "AMBIENCE": -1, "MUSIC": -1}

    i = 0
    while i < total_words:
        current_time_ms = math.ceil((i / speed_wps) * 1000)
        word = words[i]
        
        # --- 1. Check for weight modifiers ---
        if word in MODIFIER_WORDS:
            current_weight_db = MODIFIER_WORDS[word]
            print(f"[DECIDER] Word '{word}' at {current_time_ms}ms. Setting weight to: {current_weight_db}dB")
            i += 1
            continue # Go to next word

        # --- 2. Check for sound keywords (multi-word first) ---
        matched_keyword = None
        for keyword in sorted_keywords:
            kw_len = len(keyword.split())
            if i + kw_len <= total_words:
                phrase = " ".join(words[i : i + kw_len])
                if phrase == keyword:
                    matched_keyword = keyword
                    break
        
        if matched_keyword:
            audio_prompt, audio_type = SOUND_KEYWORDS[matched_keyword]
            
            print(f"[DECIDER] Keyword '{matched_keyword}' ({audio_type}) at {current_time_ms}ms.")

            # --- 3. Create the AudioCue ---
            cue = AudioCue(
                audio_prompt=audio_prompt,
                start_time_ms=current_time_ms,
                duration_ms=DEFAULT_SFX_DURATION_MS, # Default, will be changed for AMBIENCE/MUSIC
                weight_db=current_weight_db,
                audio_type=audio_type
            )
            
            # --- 4. Handle "Smart Duration" for AMBIENCE/MUSIC ---
            # We need to update the *previous* cue of the same type to stop it.
            if audio_type == "AMBIENCE" or audio_type == "MUSIC":
                last_idx = last_cue_index[audio_type]
                if last_idx != -1:
                    # Found a previous cue. Let's update its duration.
                    prev_cue = cues_to_play[last_idx]
                    prev_cue.duration_ms = current_time_ms - prev_cue.start_time_ms
                    print(f"    [DECIDER] Updating previous '{prev_cue.audio_type}' cue to end at {current_time_ms}ms.")

                # For this new cue, set its duration to play until the end (for now)
                
                cue.duration_ms = total_duration_ms - current_time_ms
                last_cue_index[audio_type] = len(cues_to_play) # Store index of this cue
            
            cues_to_play.append(cue)
            
            # Reset weight after using it
            current_weight_db = DEFAULT_WEIGHT_DB
            i += len(matched_keyword.split()) # Skip the words we just matched
        
        else:
            i += 1 # No match, move to the next word

    print(f"[DECIDER] Finished parsing. Found {len(cues_to_play)} audio cues.")
    return cues_to_play, total_duration_ms

# ==============================================================================
# 5. TOOL 1: play_audio (The Mixer / Superimposition)
# ==============================================================================
# This is the "muscle" of the operation. It takes the AudioCue list
# and uses pydub to mix them all together.

def play_audio(audio_cues: List[AudioCue], total_duration_ms: int):
    """
    Generates and mixes all audio cues into a single track (MOCKED).
    """
    print("\n[MIXER] Starting audio mixing process...")
    print(f"[MIXER] Creating silent audio canvas of {total_duration_ms}ms.")

    # --- Real Pydub Code ---
    # final_score = AudioSegment.silent(duration=total_duration_ms)
    # --- Mock Code ---
    final_score_log = [f"SILENT_CANVAS ({total_duration_ms}ms)"]


    for cue in audio_cues:
        print(f"[MIXER] Processing cue: '{cue.audio_prompt}' ({cue.audio_type})")

        # --- 1. Call Specialist Model (Mocked) ---
        specialist_func = SPECIALIST_MAP[cue.audio_type]
        mock_audio_data = specialist_func(cue.audio_prompt, cue.duration_ms)

        # --- 2. Process with Pydub (Mocked) ---
        # In real code, mock_audio_data would be an AudioSegment object.
        print(f"    [MIXER] Applying fades and volume:")
        print(f"    ... Fading In: {cue.fade_ms}ms, Fading Out: {cue.fade_ms}ms")
        print(f"    ... Adjusting Volume: {cue.weight_db}dB")

        # --- Real Pydub Code ---
        # fade_time = min(cue.fade_ms, cue.duration_ms // 2) # Don't fade longer than half the clip
        # audio_clip = mock_audio_data
        # processed_clip = audio_clip.fade_in(fade_time).fade_out(fade_time)
        # processed_clip = processed_clip + cue.weight_db # Apply volume
        
        print(f"    [MIXER] Overlaying '{cue.audio_prompt}' at {cue.start_time_ms}ms.")
        
        # --- Real Pydub Code ---
        # final_score = final_score.overlay(processed_clip, position=cue.start_time_ms)
        
        # --- Mock Code ---
        final_score_log.append(
            f"  + OVERLAY: '{cue.audio_prompt}' at {cue.start_time_ms}ms "
            f"(Duration: {cue.duration_ms}ms, Volume: {cue.weight_db}dB)"
        )

    print("\n[MIXER] === FINAL AUDIO SCORE ASSEMBLED (MOCK) ===")
    for line in final_score_log:
        print(line)
    
    # --- Real Pydub Code ---
    # print("[MIXER] Exporting to final_score.mp3...")
    # final_score.export("final_score.mp3", format="mp3")
    # print("[MIXER] Done.")
    # return final_score
    
    return final_score_log

# ==============================================================================
# 6. MAIN EXECUTION
# ==============================================================================

if __name__ == "__main__":
    
    # Your example story
    story = "Suddenly rain started so i ran to shelter where i heard loud dog barking"
    
    print("="*40)
    print(f"Processing Story: \"{story}\"")
    print("="*40 + "\n")

    # --- Run Tool 2: Decide Audio ---
    cues, total_duration = decide_audio(story, READING_SPEED_WPS)
    
    print("\n--- Generated Audio Cues ---")
    if not cues:
        print("No audio cues found.")
    for c in cues:
        print(f"  - {c}")
    print(f"Total Story Duration: {total_duration}ms")
    print("-" * 30 + "\n")
    
    # --- Run Tool 1: Play Audio ---
    play_audio(cues, total_duration)

    print("\n" + "="*40)
    print("Example 2: A more complex story")
    print("="*40 + "\n")

    story_2 = ("It was a happy day. I was walking in the forest when suddenly "
               "i felt a chill. I heard a faint dog barking. Then a loud shout!")

    cues_2, total_duration_2 = decide_audio(story_2, READING_SPEED_WPS)
    
    print("\n--- Generated Audio Cues ---")
    for c in cues_2:
        print(f"  - {c}")
    print(f"Total Story Duration: {total_duration_2}ms")
    print("-" * 30 + "\n")
    
    play_audio(cues_2, total_duration_2)

Processing Story: "Suddenly rain started so i ran to shelter where i heard loud dog barking"

[DECIDER] Starting audio decision process...
[DECIDER] Reading Speed: 2.0 words/sec
[DECIDER] Keyword 'suddenly' (MUSIC) at 0ms.
[DECIDER] Keyword 'rain' (AMBIENCE) at 500ms.
[DECIDER] Keyword 'ran' (SFX) at 2500ms.
[DECIDER] Keyword 'shelter' (AMBIENCE) at 3500ms.
    [DECIDER] Updating previous 'AMBIENCE' cue to end at 3500ms.
[DECIDER] Word 'loud' at 5500ms. Setting weight to: 6.0dB
[DECIDER] Keyword 'dog barking' (SFX) at 6000ms.
[DECIDER] Finished parsing. Found 5 audio cues.

--- Generated Audio Cues ---
  - AudioCue(audio_prompt='suspenseful stinger', start_time_ms=0, duration_ms=7000, weight_db=0.0, audio_type='MUSIC', fade_ms=500)
  - AudioCue(audio_prompt='heavy rain', start_time_ms=500, duration_ms=3000, weight_db=0.0, audio_type='AMBIENCE', fade_ms=500)
  - AudioCue(audio_prompt='running footsteps', start_time_ms=2500, duration_ms=2000, weight_db=0.0, audio_type='SFX', fade_ms=500)